In [60]:
# Import libs

from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import csv

In [63]:
# Time series CSV Urls ( Updated twice daily )
conf_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Confirmed.csv'
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Deaths.csv'
recov_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Recovered.csv'

# Building Dataframes
conf_d  =pd.read_csv(conf_url)
death_d  =pd.read_csv(death_url)
recov_d  =pd.read_csv(recov_url)



In [64]:
conf_d.head()
#death_d.head()
#recov_d.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,2/6/20 14:20,2/7/20 20:13,2/7/20 22:50,2/8/20 22:04,2/8/20 23:04,2/9/20 10:30,2/9/20 23:20,2/10/20 10:30,2/10/20 19:30,2/11/20 10:50
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,591.0,665.0,733.0,733.0,779.0,779.0,830.0,830.0,830.0,860
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,274.0,297.0,315.0,315.0,326.0,326.0,337.0,337.0,337.0,342
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,400.0,415.0,426.0,428.0,446.0,450.0,468.0,473.0,486.0,489
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,215.0,224.0,239.0,239.0,250.0,250.0,261.0,261.0,261.0,267
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,62.0,67.0,71.0,79.0,79.0,79.0,83.0,83.0,86.0,86


# Beta
## Scope:

* Total Confirmed
* Confirmed by Country/Region
* Total Deaths
* Deaths by Country/Region
* Total Recovered
* Recovered by Country/Region


In [65]:
# Total Confirmed

conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
conf_d_NumOnly['Total_Conf']=conf_d_NumOnly.iloc[:,-1]
tot_conf = conf_d_NumOnly['Total_Conf'].sum()
#conf_d_NumOnly.head()

In [67]:
tot_conf

43141

In [79]:
# Confirmed by Country
conf_d_country = conf_d_NumOnly.groupby(['Country/Region']).sum()['Total_Conf'].sort_values(ascending=False)

conf_d_country.head()
#conf_d_country.loc['Hong Kong']

Country/Region
Mainland China    42670
Others              135
Hong Kong            49
Singapore            45
Thailand             32
Name: Total_Conf, dtype: int64

## Functions
### For getting TOTAL and DF by Country

In [84]:
def get_total(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]
    tot_conf = df['Total_Conf'].sum()
    
    return tot_conf

In [85]:
def get_occr_country(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]    
    df = df.groupby(['Country/Region']).sum()['Total_Conf'].sort_values(ascending=False)
    
    return df #returns DF

In [86]:
tot_conf = get_total(conf_d)
tot_death = get_total(death_d)
tot_recov = get_total(recov_d)
print(tot_conf)
print(tot_death)
print(tot_recov)


43141
1018
4340


In [89]:
occr_conf = get_occr_country(conf_d)
occr_death = get_occr_country(death_d)
occr_recov = get_occr_country(recov_d)
#print(occr_conf)
#print(occr_death)
#print(occr_recov)


# Compiling code 

In [90]:
# Import Libs
import pandas as pd

# Functions
def get_total(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]
    tot_conf = df['Total_Conf'].sum()
    
    return tot_conf

def get_occr_country(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]    
    df = df.groupby(['Country/Region']).sum()['Total_Conf'].sort_values(ascending=False)
    
    return df #returns DF

tot_conf = get_total(conf_d)
tot_death = get_total(death_d)
tot_recov = get_total(recov_d)

occr_conf = get_occr_country(conf_d)
occr_death = get_occr_country(death_d)
occr_recov = get_occr_country(recov_d)

# Time series CSV Urls ( Updated twice daily )
conf_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Confirmed.csv'
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Deaths.csv'
recov_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Recovered.csv'

# Building Dataframes
conf_d  =pd.read_csv(conf_url)
death_d  =pd.read_csv(death_url)
recov_d  =pd.read_csv(recov_url)

# Getting Basic Variables for Dictionary
tot_conf = get_total(conf_d)
tot_death = get_total(death_d)
tot_recov = get_total(recov_d)

occr_conf = get_occr_country(conf_d)
occr_death = get_occr_country(death_d)
occr_recov = get_occr_country(recov_d)

print('Total Confirmed Cases: {}'.format(tot_conf))
print('Total Deaths: {}'.format(tot_death))
print('Total Recovery Cases: {}'.format(tot_recov))
#occr_conf.head()
#occr_death.head()
#occr_recov.head()


# Connecting to DynamoDb




43141
